In [19]:
from sqlalchemy import create_engine
from sqlalchemy.engine import URL
from urllib.parse import unquote
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float, Date, ForeignKey
from sqlalchemy.orm import relationship

In [20]:
server_name   = "localhost"
database_name = "Lab2"

connection_string = f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server_name};DATABASE={database_name};TrustServerCertificate=yes;UID=sa;PWD=123456;"
url_string        = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

print('Connecting to database using URL string:')
unquoted_url = unquote(str(url_string))
print(unquoted_url, '\n')

try:    
    engine = create_engine(url_string)
    with engine.connect() as connection:
        print(f'Successfully connected to {database_name}!')
except Exception as e:
    print('Error while connecting to database:\n')
    print(e)

Connecting to database using URL string:
mssql+pyodbc://?odbc_connect=DRIVER=ODBC+Driver+17+for+SQL+Server;SERVER=localhost;DATABASE=Lab2;TrustServerCertificate=yes;UID=sa;PWD=123456; 

Successfully connected to Lab2!


In [21]:
Session = sessionmaker(bind=engine)
session = Session()

Base = declarative_base()

/var/folders/qs/52nq18yx781_kc58fhrjc7n00000gn/T/ipykernel_21742/4204970373.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [22]:
class Book(Base):
    __tablename__ = 'Books'
    ISBN13 = Column(String, primary_key=True)
    Title = Column(String)
    Language = Column(String)
    Price = Column(Float)
    ReleaseDate = Column('Release Date', Date)
    inventory = relationship('Inventory', back_populates='book')

In [23]:
class Store(Base):
    __tablename__ = 'Stores'
    ID = Column(Integer, primary_key=True)
    StoreName = Column('Store Name', String)
    Address = Column(String)
    inventory = relationship('Inventory', back_populates='store')

In [24]:
class Inventory(Base):
    __tablename__ = 'Inventory'
    StoreID = Column(Integer, ForeignKey('Stores.ID'), primary_key=True)
    ISBN13 = Column(String, ForeignKey('Books.ISBN13'), primary_key=True)
    AmountOfBooks = Column('Amount of books', Integer)
    book = relationship('Book', back_populates='inventory')
    store = relationship('Store', back_populates='inventory')

In [25]:
def search_books(title_search):
    try:
        results = session.query(Book).filter(Book.Title.ilike(f'%{title_search}%')).all()
        
        if not results:
            print("No books found matching the search criteria.")
            return

        for book in results:
            print(f"Title: {book.Title}, ISBN: {book.ISBN13}")
            print("Stores and availability:")
            for inventory in book.inventory:
                print(f"  Store: {inventory.store.StoreName}, Amount: {inventory.AmountOfBooks}")
            print()

    except Exception as e:
        print(f"An error occurred: {e}")

In [29]:
while True:
    title_search = input("Enter a book title to search (or 'exit' to quit): ")
    if title_search.lower() == 'exit':
        break
    search_books(title_search)

Title: The Secret Garden, ISBN: 978-0-316-76948-0
Stores and availability:
  Store: Booktopia, Amount: 48
  Store: Readmore Books, Amount: 35
  Store: Novel Ideas, Amount: 0
  Store: The Reading Room, Amount: 81
  Store: Page Turners, Amount: 106

Title: Secrets of the Past, ISBN: 978-1-5040-5104-2
Stores and availability:
  Store: Booktopia, Amount: 71
  Store: Readmore Books, Amount: 25
  Store: Novel Ideas, Amount: 91
  Store: The Reading Room, Amount: 0
  Store: Page Turners, Amount: 0

